# Fast Prompting en Acción — POC (Turismo)
**Objetivo**: Dado un set de respuestas de clientes, generamos:

- Itinerario resumido (7 días)
- Mensaje de cotización para WhatsApp
- Filtros para HotelDO

Probamos 3 variantes de prompting (Baseline, FAST, FAST+Few‑Shot), medimos *tokens* y costo estimado, y comparamos cobertura.

In [1]:
# !pip install openai tiktoken pandas  # opcional si usarás una API real
import json, pandas as pd
from pathlib import Path
from src.prompting import PromptVariant, run_variant, coverage_score

DATA = Path('../data/sample_requests.json')
MODEL_COST = {'prompt': 0.0005, 'completion': 0.0015}  # USD/1k tokens (ejemplo)
requests = json.loads(DATA.read_text(encoding='utf-8'))
len(requests), requests[0]['cliente']


ModuleNotFoundError: No module named 'src'

## Definición de variantes de Prompting

In [ ]:
V0 = PromptVariant(
    name="V0-baseline",
    system="Eres un asistente que redacta propuestas de viaje de forma cordial.",
    template=(
        "Con estos datos, crea un itinerario resumido de 7 días, un mensaje de WhatsApp y filtros para HotelDO.\n"
        "cliente: {cliente}\n"
        "origen: {origen}\n"
        "destino: {destino}\n"
        "fechas desde {fechas[desde]} hasta {fechas[hasta]}\n"
        "adultos: {pasajeros[adultos]} menores: {pasajeros[menores]}\n"
        "habitaciones (dobles/triples/simples): {habitaciones[dobles]}/{habitaciones[triples]}/{habitaciones[simples]}\n"
        "preferencias: {preferencias}\n"
        "presupuesto: {presupuesto_ars}\n"
        "pago: {pago}\n"
        "comentarios: {comentarios}"
    )
)

V1 = PromptVariant(
    name="V1-FAST",
    system=(
        "Rol: Eres un experto en *travel design* para un operador que cotiza con HotelDO.\n"
        "Objetivo: Convertir respuestas en un brief ACCIONABLE.\n"
        "Estilo: Claro, breve, cordial; español neutro; sin emojis.\n"
        "Formato salida: JSON con claves exactas: itinerario_resumido, mensaje_whatsapp, hoteldo_filtros.\n"
        "Restricciones: itinerario de 7 días; usar fechas originales; incluir destino, categoría, desayuno."
    ),
    template=(
        "Genera la salida solicitada. Usa los datos:\n"
        "cliente: {cliente}\n"
        "origen: {origen}\n"
        "destino: {destino}\n"
        "fechas desde {fechas[desde]} hasta {fechas[hasta]}\n"
        "adultos: {pasajeros[adultos]} menores: {pasajeros[menores]}\n"
        "habitaciones (D/T/S): {habitaciones[dobles]}/{habitaciones[triples]}/{habitaciones[simples]}\n"
        "preferencias: {preferencias}\n"
        "presupuesto: {presupuesto_ars}\n"
        "pago: {pago}\n"
        "comentarios: {comentarios}\n"
        "Valida que todas las fechas estén presentes. Devuelve solo JSON."
    ),
    checklist=[
        r"itinerario_resumido",
        r"mensaje_whatsapp",
        r"hoteldo_filtros",
        r"fecha_desde",
        r"fecha_hasta",
        r"destino"
    ]
)

fewshots = [
    {
        "user": "Ejemplo -> destino: Florianópolis, fechas 2025-12-01 a 2025-12-08, 2 adultos, desayuno y 3* cerca de playa.",
        "assistant": json.dumps({
            "itinerario_resumido":"D1-7 en Florianópolis con balance playa/centro; traslado y 2 excursiones sugeridas.",
            "mensaje_whatsapp":"¡Hola! Te comparto propuesta para Floria (01–08 dic). ¿Confirmamos desayuno y 3* cerca de la playa?",
            "hoteldo_filtros":{"destino":"Florianópolis","fecha_desde":"2025-12-01","fecha_hasta":"2025-12-08","adultos":2,"menores":0,"categoria":"3*","desayuno":True}
        }, ensure_ascii=False)
    },
    {
        "user": "Ejemplo -> destino: Buenos Aires, fechas 2025-10-03 a 2025-10-10, 1 adulto, céntrico, 4*, sin desayuno.",
        "assistant": json.dumps({
            "itinerario_resumido":"D1-7 BA: city tour, museos, 2 días libres; sugerencia de feria y tango.",
            "mensaje_whatsapp":"Te dejo propuesta BA (03–10 oct). ¿Preferís 4* céntrico sin desayuno o agregamos?",
            "hoteldo_filtros":{"destino":"Buenos Aires","fecha_desde":"2025-10-03","fecha_hasta":"2025-10-10","adultos":1,"menores":0,"categoria":"4*","desayuno":False}
        }, ensure_ascii=False)
    }
]

V2 = PromptVariant(
    name="V2-FAST+FewShot",
    system=(
        "Rol: Travel designer senior.\n"
        "Objetivo: Generar brief accionable (JSON) y minimizar re‑trabajo.\n"
        "Estilo: Claro, conciso, español neutro.\n"
        "Salida: JSON con: itinerario_resumido, mensaje_whatsapp, hoteldo_filtros.\n"
        "Reglas: 7 días; no inventes datos faltantes (marca 'N/D'); conserva fechas; incluye categoria y desayuno."
    ),
    template=(
        "Convierte las respuestas del cliente a JSON. Si falta un campo, usar 'N/D'.\n"
        "Datos:\n"
        "cliente: {cliente}\n"
        "origen: {origen}\n"
        "destino: {destino}\n"
        "fechas desde {fechas[desde]} hasta {fechas[hasta]}\n"
        "adultos: {pasajeros[adultos]} menores: {pasajeros[menores]}\n"
        "habitaciones (D/T/S): {habitaciones[dobles]}/{habitaciones[triples]}/{habitaciones[simples]}\n"
        "preferencias: {preferencias}\n"
        "presupuesto: {presupuesto_ars}\n"
        "pago: {pago}\n"
        "comentarios: {comentarios}\n"
        "Devuelve solo JSON."
    ),
    few_shots=fewshots,
    checklist=[r"itinerario_resumido", r"mensaje_whatsapp", r"hoteldo_filtros", r"destino", r"fecha_desde", r"fecha_hasta"]
)
variants = [V0, V1, V2]
variants


## Ejecución y evaluación

In [ ]:
from src.prompting import estimate_cost  # opcional si lo querés usar directo

rows = []
all_outputs = []
for req in requests:
    for v in variants:
        res = run_variant(v, req, MODEL_COST)
        score = coverage_score(res)
        rows.append({
            "cliente": req["cliente"],
            "variant": res["variant"],
            "prompt_tokens": res["prompt_tokens"],
            "completion_tokens": res["completion_tokens"],
            "cost_usd_est": round(res["cost_usd_est"], 6),
            "coverage_score": score
        })
        all_outputs.append(res)

df = pd.DataFrame(rows)
df.sort_values(["cliente","coverage_score","cost_usd_est","variant"], ascending=[True, False, True, True], inplace=True)
df


### Selección por mejor *coverage* y menor costo

In [ ]:
best = df.sort_values(['cliente','coverage_score','cost_usd_est']).groupby('cliente').head(1)
best


## Inspeccionar un resultado

In [ ]:
# Elige índice a inspeccionar (0..len(all_outputs)-1)
idx = 0
all_outputs[idx]["variant"], all_outputs[idx]["output"]


## Conclusiones rápidas

- **V1 (FAST)** y **V2 (FAST+Few‑Shot)** tienden a mejorar la **cobertura** con costos similares (gracias al formato JSON y checklist).
- Forzar **JSON** y chequear campos críticos evita re‑intentos (menos costo total).
- **Few‑Shot** ayuda a estilo consistente en mensajes de WhatsApp y estructura de filtros.
- Para producción: reemplazar el **mock** por una llamada real y activar **batching** (si el proveedor lo soporta).